# keras tuner

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [2]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [3]:
!pip install -q -U keras-tuner

In [4]:
import kerastuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Load the data.

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [10]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 28s]
val_accuracy: 0.8944166898727417

Best val_accuracy So Far: 0.8944166898727417
Total elapsed time: 00h 06m 04s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6147 - accuracy: 0.7839 - val_loss: 0.4014 - val_accuracy: 0.8539
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3753 - accuracy: 0.8655 - val_loss: 0.3886 - val_accuracy: 0.8607
Epoch 3/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3298 - accuracy: 0.8782 - val_loss: 0.3729 - val_accuracy: 0.8648
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3069 - accuracy: 0.8860 - val_loss: 0.3373 - val_accuracy: 0.8801
Epoch 5/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2812 - accuracy: 0.8933 - val_loss: 0.3202 - val_accuracy: 0.8818
Epoch 6/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2664 - accuracy: 0.8997 - val_loss: 0.3141 - val_accuracy: 0.8860
Epoch 7/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2533 - accuracy: 0.9065 - val_loss: 0.3128 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_test, label_test, epochs=best_epoch)

Epoch 1/13
313/313 [==============================] - 1s 2ms/step - loss: 0.9074 - accuracy: 0.6859
Epoch 2/13
313/313 [==============================] - 1s 2ms/step - loss: 0.5113 - accuracy: 0.8173
Epoch 3/13
313/313 [==============================] - 1s 2ms/step - loss: 0.4458 - accuracy: 0.8333
Epoch 4/13
313/313 [==============================] - 1s 2ms/step - loss: 0.3891 - accuracy: 0.8620
Epoch 5/13
313/313 [==============================] - 1s 2ms/step - loss: 0.3666 - accuracy: 0.8620
Epoch 6/13
313/313 [==============================] - 1s 2ms/step - loss: 0.3559 - accuracy: 0.8717
Epoch 7/13
313/313 [==============================] - 1s 2ms/step - loss: 0.3366 - accuracy: 0.8733
Epoch 8/13
313/313 [==============================] - 1s 2ms/step - loss: 0.3138 - accuracy: 0.8820
Epoch 9/13
313/313 [==============================] - 1s 2ms/step - loss: 0.2926 - accuracy: 0.8904
Epoch 10/13
313/313 [==============================] - 1s 2ms/step - loss: 0.2771 - accuracy: 0.8947

To finish this tutorial, evaluate the hypermodel on the test data.

In [13]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.2313 - accuracy: 0.9147
[test loss, test accuracy]: [0.23128622770309448, 0.9146999716758728]
